In [51]:
import pyspark
from pyspark.sql.functions import col,count,sum,abs,round

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Joins').getOrCreate()
spark

In [8]:
df1 = spark.read.csv(r'C:\Users\Leon\Downloads\covid\data\country_wise_latest.csv',header=True,inferSchema=True)
df1.show()


+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|               3.5|                69.49|                  5.04|              35526|          737|             2.07|Eastern Mediterra...|
|            Albania|     4880|   144|     2745|  1991|      117|         6|           6

In [54]:
df2 = spark.read.csv(r'C:\Users\Leon\Downloads\covid\data\worldometer_data.csv',header=True,inferSchema=True)
df2.show()
df3 = spark.read.csv(r'C:\Users\Leon\Downloads\covid\data\country_vaccinations.csv',header=True,inferSchema=True)
df3.show()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|    NULL|     162804|     NULL|       2576668|        NULL|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    NULL|      98644|     NULL|       2047660|        NULL|     771258|         

Using PySpark, perform an inner join between the two datasets on Country/Region, and select countries where the confirmed cases from country_wise_latest and total cases from worldometer_data do not match. List those countries along with both values

In [12]:
joined_df=df1.join(df2,on = "Country/Region",how = "inner")
joined_df=joined_df.filter(col("Confirmed")!= col("TotalCases")).select("Country/Region", "Confirmed", "TotalCases")
joined_df.show()

+--------------+---------+----------+
|Country/Region|Confirmed|TotalCases|
+--------------+---------+----------+
|        Brazil|  2442375|   2917562|
|         India|  1480073|   2025409|
|        Russia|   816680|    871894|
|  South Africa|   452529|    538184|
|        Mexico|   395489|    462690|
|          Peru|   389717|    455409|
|         Chile|   347923|    366671|
|      Colombia|   257101|    357710|
|         Spain|   272421|    354530|
|          Iran|   293606|    320117|
|  Saudi Arabia|   268934|    284226|
|      Pakistan|   274289|    281863|
|    Bangladesh|   226225|    249651|
|         Italy|   246286|    249204|
|        Turkey|   227019|    237265|
|     Argentina|   167416|    228195|
|       Germany|   207112|    215210|
|        France|   220352|    195633|
|          Iraq|   112585|    140603|
|   Philippines|    82040|    119460|
+--------------+---------+----------+
only showing top 20 rows



Perform a left join from country_wise_latest to worldometer_data on Country/Region. Identify countries that exist in the country_wise_latest but have no match in worldometer_data (i.e., null values).

In [19]:
joined_df = df1.join(df2,on="Country/Region",how = "left")
joined_df = joined_df.filter(col("TotalCases").isNull()).select("Country/Region").show()

+--------------------+
|      Country/Region|
+--------------------+
|              Brunei|
|               Burma|
|Central African R...|
|               China|
| Congo (Brazzaville)|
|    Congo (Kinshasa)|
|       Cote d'Ivoire|
|            Holy See|
|              Kosovo|
|Saint Vincent and...|
|         South Korea|
|             Taiwan*|
|                  US|
|United Arab Emirates|
|      United Kingdom|
|  West Bank and Gaza|
+--------------------+



Join the two datasets on Country/Region, and group the data by Continent. Then, compute the total number of deaths (from Deaths in country_wise_latest) for each continent. Return the top 3 continents with the highest death counts.

In [33]:
joined_df=df1.join(df2,on="Country/Region",how = "inner")
joined_df=joined_df.groupBy('Continent').agg(sum("Deaths").alias("TotalDeaths")).orderBy(col("TotalDeaths").desc()).show(3)


+-------------+-----------+
|    Continent|TotalDeaths|
+-------------+-----------+
|       Europe|     155775|
|South America|     135506|
|         Asia|      85740|
+-------------+-----------+
only showing top 3 rows



Join the two datasets and calculate the difference in recovery rate between them:

From country_wise_latest, use Recovered / Confirmed.

From worldometer_data, use TotalRecovered / TotalCases.

List the top 5 countries with the highest difference in recovery rates.





In [53]:
joined_df= df1.join(df2,on = "Country/Region",how = "inner")
joined_df = joined_df.filter((col("Confirmed") > 0) & (col("TotalCases") > 0))
joined_df=joined_df.withColumn("Recovery_rate_country",round(col("Recovered")/col("Confirmed"),2))
joined_df=joined_df.withColumn("Recovery_rate_worldometer",round(col("TotalRecovered")/col("TotalCases"),2))
joined_df=joined_df.withColumn("Recovery_rate_diff",round(abs(col("Recovery_rate_country")-col("Recovery_rate_worldometer")),2))
joined_df.select("Country/Region","Recovery_rate_country","Recovery_rate_worldometer","Recovery_rate_diff").orderBy(col("Recovery_rate_diff").desc()).show(5)


+--------------+---------------------+-------------------------+------------------+
|Country/Region|Recovery_rate_country|Recovery_rate_worldometer|Recovery_rate_diff|
+--------------+---------------------+-------------------------+------------------+
|   Timor-Leste|                  0.0|                     0.96|              0.96|
|        Canada|                  0.0|                     0.87|              0.87|
|    Seychelles|                 0.34|                     0.98|              0.64|
|        Serbia|                  0.0|                     0.51|              0.51|
|    Mozambique|                  0.0|                     0.38|              0.38|
+--------------+---------------------+-------------------------+------------------+
only showing top 5 rows



Store the results in postgres

In [59]:
postgres_url = "jdbc:postgresql://localhost:5432/testdb"
properties = {
    "user": "postgres",
    "password": "March#123",
    "driver": "org.postgresql.Driver"
}

df1.write.jdbc(url=postgres_url, table="covid", mode="overwrite", properties=properties)


Py4JJavaError: An error occurred while calling o768.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$Lambda$3942/13941412.apply(Unknown Source)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spa